<a href="https://colab.research.google.com/github/AltagiAbdallah/Strategic-Workforce-Retention/blob/main/Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import os

# This mounts our Google Drive to the Colab notebook
drive.mount('/content/drive')

Mounted at /content/drive


**Step 1:Load Data**

In [ ]:
#load the dataset
df = pd.read_csv("/content/drive/MyDrive/Data science projects /DA/Datasets/HR_Analytics.csv.csv")

#display the first few rows
df.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


**Step 2: Data Cleaning (Removing Noise)**

In [ ]:
# Check for missing values
print("Missing Values:\n", df.isnull().sum().sum())

# Drop useless columns
# EmployeeCount: Always 1
# Over18: Always 'Y'
# StandardHours: Always 80
cols_to_drop = ['EmployeeCount', 'Over18', 'StandardHours']
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("Useless columns dropped.")

Missing Values:
 0
Useless columns dropped.


**Step 3: Feature Engineering**

In [ ]:
# --- Data Cleaning & Feature Engineering ---

# 1. Drop useless columns
df.drop(columns=['EmployeeCount', 'Over18', 'StandardHours'], inplace=True, errors='ignore')

# 2. Age Grouping
def classify_age(age):
    if age < 25: return '18-24 (Gen Z)'
    elif age < 35: return '25-34 (Young Professionals)'
    elif age < 45: return '35-44 (Mid-Career)'
    elif age < 55: return '45-54 (Senior)'
    else: return '55+ (Veteran)'

df['AgeGroup'] = df['Age'].apply(classify_age)

# 3. Tenure Grouping
def classify_tenure(years):
    if years <= 2: return '0-2 Years (New Hires)'
    elif years <= 5: return '3-5 Years (Mid Tenure)'
    elif years <= 10: return '6-10 Years (Loyal)'
    else: return '10+ Years (Veterans)'

df['TenureGroup'] = df['YearsAtCompany'].apply(classify_tenure)

# 4. Attrition Numeric
df['Attrition_Count'] = df['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)

# 5. Flight Risk Tag
def flight_risk(row):
    if row['JobSatisfaction'] <= 2 and row['OverTime'] == 'Yes':
        return 'High Risk'
    else:
        return 'Normal'

df['Flight_Risk_Status'] = df.apply(flight_risk, axis=1)

print("Data processing complete.")
df.head()

Data processing complete.


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,AgeGroup,TenureGroup,Attrition_Count,Flight_Risk_Status
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,2,...,0,1,6,4,0,5,35-44 (Mid-Career),6-10 Years (Loyal),1,Normal
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,2,3,...,3,3,10,7,1,7,45-54 (Senior),6-10 Years (Loyal),0,Normal
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,4,...,3,3,0,0,0,0,35-44 (Mid-Career),0-2 Years (New Hires),1,Normal
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,5,4,...,3,3,8,7,3,0,25-34 (Young Professionals),6-10 Years (Loyal),0,Normal
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,7,1,...,3,3,2,2,2,2,25-34 (Young Professionals),0-2 Years (New Hires),0,Normal


In [ ]:
# We use os.path.dirname to get the folder path from our input file_path
file_path = '/content/drive/MyDrive/Data science projects /DA/Datasets/HR_Analytics.csv.csv'
output_path = os.path.dirname(file_path) + '/HR_Analytics_Cleaned_for_Dashboard.csv'

df.to_csv(output_path, index=False)

print(f"File saved successfully to Drive at:\n{output_path}")

File saved successfully to Drive at:
/content/drive/MyDrive/Data science projects /DA/Datasets/HR_Analytics_Cleaned_for_Dashboard.csv
